# The feedforward neural network model for learning PageRank based on the order 6 CDF matrix representation

In [1]:
import time
from mygraph import MyGraph
from helpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("fb_media_pages_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([4.26055693e-05])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 24000
target_pagerank[0]

array([1.02253366])

In [5]:
target_pagerank.shape

(27917, 1)

In [6]:
# the order 6 CDF matrix as the feature set
matrix_1 = np.load("fb_media_pages_CDF_matrix_r1_3_sta1_max70_rad6.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [7]:
matrix_1.shape

(27917, 6, 23)

In [8]:
matrix_1[111]

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, 4.00000e-01, 4.00000e-01,
        0.00000e+00, 2.00000e-01, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00],
       [2.50000e-01, 0.00000e+00, 8.33330e-02, 8.33330e-02, 0.00000e+00,
        0.00000e+00, 8.33330e-02, 1.66667e-01, 1.66667e-01, 0.00000e+00,
        8.33330e-02, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 8.33330e-02, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00],
       [3.52940e-02, 2.05880e-02, 1.17650e-02, 6.17650e-02, 9.41180e-02,
        1.17647e-01, 1.17647e-01, 1.41176e-01, 1.55882e-01, 7.94120e-02,
        4.70590e-02, 3.23530e-02, 2.64710e-02, 1.17650e-02, 1.47060e-02,
        8.82400e-03, 1.17650e-02, 2.94100e-03, 0.00000e+00, 5.88200e-03,
        0.00000e+00, 2.94100

In [9]:
matrix_1 = matrix_1.reshape(27917,-1)
matrix_1.shape

(27917, 138)

# Converting the data into pytorch tenors

In [10]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([27917, 138]), torch.Size([27917, 1]))

In [11]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[1.0225],
        [0.9398],
        [0.8766],
        ...,
        [0.4565],
        [0.9406],
        [0.9156]], dtype=torch.float64, requires_grad=True)

In [12]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 7917
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([20000, 138]),
 torch.Size([7917, 138]),
 torch.Size([20000, 1]),
 torch.Size([7917, 1]))

## A function for dividing train data into batches

In [13]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [14]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [15]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=138, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [16]:
# Number of parameters
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

55200
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
549993


In [17]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [18]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [19]:
losses = training_loop(n_epochs=2000,
                  batch_size=500,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   1.06266401112,    TEST LOSS:   0.76792508364
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.48382532597,    TEST LOSS:   0.49373704195
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.33693018556,    TEST LOSS:   0.42955383658
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.25619801879,    TEST LOSS:   0.38823586702
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.20915637910,    TEST LOSS:   0.36380755901
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.16890974343,    TEST LOSS:   0.34813657403
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.13848783076,    TEST LOSS:   0.32267442346
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.11610569805,    TEST LOSS:   0.31220379472
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [20]:
# Saving the train losses of pagerank as a numpy array into a file
np.save("ffnn_fb_media_pagerank_train_losses_005_using_CDF.npy", losses[0], allow_pickle=False, fix_imports=True)

In [21]:
# Saving the test losses of pagerank as a numpy array into a file
np.save("ffnn_fb_media_pagerank_test_losses_005_using_CDF.npy", losses[1], allow_pickle=False, fix_imports=True)

In [22]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,138)
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%50 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------------------------------------")

target 0.5887202784849995,    prediction: 0.5420475006103516
index 0:       inaccuracy: 7.928%
-----------------------------------------------------------------
target 0.8238909976897885,    prediction: 0.638022243976593
index 50:       inaccuracy: 22.56%
-----------------------------------------------------------------
target 0.3000673203111709,    prediction: 0.40677565336227417
index 100:       inaccuracy: 35.561%
-----------------------------------------------------------------
target 0.9013019035860877,    prediction: 0.9772288799285889
index 150:       inaccuracy: 8.424%
-----------------------------------------------------------------
target 0.43171485807832555,    prediction: 0.4458550214767456
index 200:       inaccuracy: 3.275%
-----------------------------------------------------------------
target 2.439106569039261,    prediction: 2.6797776222229004
index 250:       inaccuracy: 9.867%
-----------------------------------------------------------------
target 4.130724373307258

target 1.713583881247917,    prediction: 2.548166036605835
index 2850:       inaccuracy: 48.704%
-----------------------------------------------------------------
target 2.3125585250718164,    prediction: 2.125861883163452
index 2900:       inaccuracy: 8.073%
-----------------------------------------------------------------
target 0.5386156905748365,    prediction: 1.3621153831481934
index 2950:       inaccuracy: 152.892%
-----------------------------------------------------------------
target 0.25284940900838115,    prediction: 0.3326976001262665
index 3000:       inaccuracy: 31.579%
-----------------------------------------------------------------
target 0.3199380191880695,    prediction: 0.5361142158508301
index 3050:       inaccuracy: 67.568%
-----------------------------------------------------------------
target 0.6885393109616987,    prediction: 0.6003387570381165
index 3100:       inaccuracy: 12.81%
-----------------------------------------------------------------
target 0.9223

target 1.2487728477602384,    prediction: 1.2441984415054321
index 5550:       inaccuracy: 0.366%
-----------------------------------------------------------------
target 0.21156520430167206,    prediction: 0.29762646555900574
index 5600:       inaccuracy: 40.678%
-----------------------------------------------------------------
target 0.5380586812171619,    prediction: 0.36627721786499023
index 5650:       inaccuracy: 31.926%
-----------------------------------------------------------------
target 0.8437575031968626,    prediction: 0.647930920124054
index 5700:       inaccuracy: 23.209%
-----------------------------------------------------------------
target 0.30096102613751907,    prediction: 0.3620169162750244
index 5750:       inaccuracy: 20.287%
-----------------------------------------------------------------
target 0.30659636712025634,    prediction: 0.37402787804603577
index 5800:       inaccuracy: 21.994%
-----------------------------------------------------------------
target

In [23]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  28.44


In [24]:
torch.save(model, "trained_FFNN_model_fb_media_pagerank_005_using_CDF.pt")